***GENERATED CODE FOR datapipetest1 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def replaceValues(df, functionsData, applyOn):
    for columnData in applyOn:
        for functionData in functionsData:
            if functionData['replaceType'] == 'value':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        toReplace, functionData['ReplaceWith'])
            elif functionData['replaceType'] == 'none':
                if functionData['replaceValueType'] == 'min':
                    minValue = df[columnData['columnName']].min().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", minValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(minValue)
                elif functionData['replaceValueType'] == 'max':
                    maxValue = df[columnData['columnName']].max().compute()
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].replace("", maxValue)
                    df[columnData['columnName']
                       ] = df[columnData['columnName']].fillna(maxValue)
                else:
                    df[columnData['columnName']] = df[columnData['columnName']].replace(
                        "", functionData['ReplaceWith'])
                    df[columnData['columnName']] = df[columnData['columnName']].fillna(
                        functionData['ReplaceWith'])
    return df


def renameColumns(df, functionsData, applyOn):
    for functionData in functionsData:
        df = df.rename(
            columns={functionData['oldName']: functionData['newName']})
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***CONNECTOR FUNCTIONS TO WRITE DATA.***

In [ ]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, path):
        df.to_csv("/dbfs" + eval(config)['url'], header=True, index=False)
        return True


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run datapipetest1Hooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Demodb2020', 'optionalDB': 'Demodb2020', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT sub2.ChargeDetailID,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,\r\n                CASE WHEN sub2.pdiagnosis1 IS NOT NULL THEN sub2.pdiagnosis1 ELSE 'XXXX' END as pdiagnosis1,\r\n                CASE WHEN sub2.pdiagnosis2 IS NOT NULL THEN sub2.pdiagnosis2 ELSE 'XXXX' END as pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,\r\n\t\t\t\tDATEDIFF(day, MIN(sub2.Chargedate), MIN(sub1.PaymentDate)) as days_to_first_post,\r\n\t\t\t\tSub2.PayerId\r\nFROM \r\n(\r\n\t\tSELECT \r\n\t\tDISTINCT cd.ChargeDetailID, Transactiondate as Chargedate,\r\n\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t\tp.Gender, cd.DateOfEntry, cd.DateOfService,\r\n\t\tpp.ProviderID,\r\n\t\tcptc.CPTCode,\r\n\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\td.DepartmentCategory,\r\n\t\ts.SpecialityCategory,\r\n\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID\r\n\t\tfrom Transactions t\r\n\t\tInner join chargedetail cd on t.Chargedetailid = cd.chargedetailid\r\n\t\tleft join transactiontype tt on t.TransactionTypeID=tt.TransactionTypeID\r\n\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\tWHERE TransactiontypeCategory = 'Charges'\r\n\t\tand t.DateOfEntry between '12/1/2020' and '12/31/2020'  \r\n) as sub2\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, TransactionDate as PaymentDate,\r\n\t\t\t t.DateOfEntry,\r\n\t\t\t count(*) as total_transactions\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tWHERE t.Amount > 0.1 \r\n\t\tand tt.TransactionTypeCategory = 'Payments' and transactiondate is not null \r\n\t\tgroup by t.DateOfEntry, t.ChargeDetailID,TransactionDate\r\n) as sub1\r\n on sub1.cid = sub2.ChargeDetailID\r\n where sub2.Gender is not NULL\r\n  Group by sub2.ChargeDetailID,  ChargeDate,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,sub2.pdiagnosis1, sub2.pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,sub1.PaymentDate,\r\n\t\t\t\tSub2.PayerId", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run datapipetest1Hooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(sqlserver,spark,json.dumps( {"url": null, "source_attributes": {"host": "wsc-sqlserver1.database.windows.net", "port": "1433", "password": "G5gTW5rnKVeAZWXm", "database": "Demodb2020", "optionalDB": "Demodb2020", "user": "Numtra", "qtype": "query", "query": "SELECT DISTINCT sub2.ChargeDetailID,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,\r\n                CASE WHEN sub2.pdiagnosis1 IS NOT NULL THEN sub2.pdiagnosis1 ELSE 'XXXX' END as pdiagnosis1,\r\n                CASE WHEN sub2.pdiagnosis2 IS NOT NULL THEN sub2.pdiagnosis2 ELSE 'XXXX' END as pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,\r\n\t\t\t\tDATEDIFF(day, MIN(sub2.Chargedate), MIN(sub1.PaymentDate)) as days_to_first_post,\r\n\t\t\t\tSub2.PayerId\r\nFROM \r\n(\r\n\t\tSELECT \r\n\t\tDISTINCT cd.ChargeDetailID, Transactiondate as Chargedate,\r\n\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t\tp.Gender, cd.DateOfEntry, cd.DateOfService,\r\n\t\tpp.ProviderID,\r\n\t\tcptc.CPTCode,\r\n\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\td.DepartmentCategory,\r\n\t\ts.SpecialityCategory,\r\n\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID\r\n\t\tfrom Transactions t\r\n\t\tInner join chargedetail cd on t.Chargedetailid = cd.chargedetailid\r\n\t\tleft join transactiontype tt on t.TransactionTypeID=tt.TransactionTypeID\r\n\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\tWHERE TransactiontypeCategory = 'Charges'\r\n\t\tand t.DateOfEntry between '12/1/2020' and '12/31/2020'  \r\n) as sub2\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, TransactionDate as PaymentDate,\r\n\t\t\t t.DateOfEntry,\r\n\t\t\t count(*) as total_transactions\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tWHERE t.Amount > 0.1 \r\n\t\tand tt.TransactionTypeCategory = 'Payments' and transactiondate is not null \r\n\t\tgroup by t.DateOfEntry, t.ChargeDetailID,TransactionDate\r\n) as sub1\r\n on sub1.cid = sub2.ChargeDetailID\r\n where sub2.Gender is not NULL\r\n  Group by sub2.ChargeDetailID,  ChargeDate,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,sub2.pdiagnosis1, sub2.pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,sub1.PaymentDate,\r\n\t\t\t\tSub2.PayerId", "dbtype": "mssql", "is_header": "Use Header Line", "server_url": "/numtraPlatform/NumtraPlatformV2/uploads/platform/"}, "DataPrepFile": null, "data_source": "mssql", "startListenerOnly": 1, "FilePath": "/FileStore/platform/extra/606ee6c4a3e14e5efe65fcc71617882049/0part.csv", "requestRatio": 28.0, "totalRows": 13919, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 15, "numberOfRows": 13919, "duplicateRowCount": 0, "stats": [{"column": "ChargeDetailID", "alias": "ChargeDetailID", "generated": 0, "type": "String", "max": "FFFFCA42-A3B8-448D-B94D-DD87D0C5C456", "min": "0008EE4F-60AB-4CC9-BF33-2BFC979E6942", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Age", "alias": "Age", "generated": 0, "type": "real", "max": 108.0, "min": 3.0, "mean": 70.57101803290466, "missing": 0.0, "stddev": 14.84}, {"column": "Gender_New", "alias": "Gender_New", "generated": 1, "type": "String", "max": "M", "min": "F", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DateOfEntry", "alias": "DateOfEntry", "generated": 0, "type": "String", "max": "2020-12-31T00:00:00.000Z", "min": "2020-12-02T00:00:00.000Z", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DateOfService", "alias": "DateOfService", "generated": 0, "type": "String", "max": "2020-12-29T00:00:00.000Z", "min": "2020-01-11T00:00:00.000Z", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "ProviderID", "alias": "ProviderID", "generated": 0, "type": "String", "max": "CBBD4925-1757-45BC-A853-1C2CA203B9E3", "min": "03151B5D-685B-4211-B01F-7BBB1C4A8A49", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "CPTCode", "alias": "CPTCode", "generated": 0, "type": "String", "max": "J9035", "min": "65880", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "pdiagnosis1", "alias": "pdiagnosis1", "generated": 0, "type": "String", "max": "Z98.890", "min": "A51.43", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "pdiagnosis2", "alias": "pdiagnosis2", "generated": 0, "type": "String", "max": "Z98.890", "min": "A51.43", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DepartmentCategory", "alias": "DepartmentCategory", "generated": 0, "type": "String", "max": "Pulmonary", "min": "Cardiology", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "SpecialityCategory", "alias": "SpecialityCategory", "generated": 0, "type": "String", "max": "Ophtamology", "min": "Ophtamology", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "days_to_first_post", "alias": "days_to_first_post", "generated": 0, "type": "real", "max": 31.0, "min": -722.0, "mean": -453.33723687046484, "missing": 0.0, "stddev": 337.23}, {"column": "PayerId", "alias": "PayerId", "generated": 0, "type": "String", "max": "FF58D17E-F6AC-4390-999A-AE5378E83883", "min": "012FD8A9-37BE-4D13-B808-7E6742223444", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Age_Replaced", "alias": "Age_Replaced", "generated": 1, "type": "real", "max": 105.0, "min": 3.0, "mean": 70.56871901717078, "missing": 0.0, "stddev": 14.83}, {"column": "Gender_Replaced", "alias": "Gender_Replaced", "generated": 1, "type": "String", "max": "Male", "min": "F", "mean": "", "missing": 0.0, "stddev": ""}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "replaceBy", "applyOn": [{"columnName": "Age", "type": "real", "min": "3.0", "max": "108.0", "mean": "14.8"}], "functionsData": [{"replaceType": "value", "toReplace": "108", "replaceValueType": "byValue", "ReplaceWith": "100", "asNewColumn": 1, "newColumnName": ""}]}, {"functionName": "replaceBy", "applyOn": [{"columnName": "Gender", "type": "String", "min": "F", "max": "M", "mean": "-"}], "functionsData": [{"replaceType": "value", "toReplace": "F", "replaceValueType": "byValue", "ReplaceWith": "Fem", "asNewColumn": 1, "newColumnName": ""}]}, {"functionName": "replaceBy", "applyOn": [{"columnName": "Gender", "type": "String", "min": "F", "max": "M", "mean": "-"}], "functionsData": [{"replaceType": "value", "toReplace": "F", "replaceValueType": "byValue", "ReplaceWith": "Fem", "asNewColumn": 1, "newColumnName": ""}, {"replaceType": "value", "toReplace": "M", "replaceValueType": "byValue", "ReplaceWith": "Male", "asNewColumn": 1, "newColumnName": ""}]}, {"functionName": "editname", "applyOn": [{"columnName": "Gender", "type": "String", "min": "F", "max": "M", "mean": "-"}], "functionsData": [{"oldName": "Gender", "newName": "Gender_New", "asNewColumn": 0, "column": {"columnName": "Gender", "type": "String", "min": "F", "max": "M", "mean": "-"}}]}], "functionChanges": [{"columnName": "Age", "functionName": "Replace Outliers", "Type": "real", "Parameters": [{"replaceType": "value", "toReplace": "108", "replaceValueType": "byValue", "ReplaceWith": "100", "asNewColumn": 1, "newColumnName": ""}]}, {"columnName": "Gender", "functionName": "Replace Outliers", "Type": "String", "Parameters": [{"replaceType": "value", "toReplace": "F", "replaceValueType": "byValue", "ReplaceWith": "Fem", "asNewColumn": 1, "newColumnName": ""}]}, {"columnName": "Gender", "functionName": "Replace Outliers", "Type": "String", "Parameters": [{"replaceType": "value", "toReplace": "F", "replaceValueType": "byValue", "ReplaceWith": "Fem", "asNewColumn": 1, "newColumnName": ""}, {"replaceType": "value", "toReplace": "M", "replaceValueType": "byValue", "ReplaceWith": "Male", "asNewColumn": 1, "newColumnName": ""}]}, {"columnName": "Gender", "functionName": "Edit Column Name", "Type": "String", "Parameters": [{"oldName": "Gender", "newName": "Gender_New", "asNewColumn": 0, "column": {"columnName": "Gender", "type": "String", "min": "F", "max": "M", "mean": "-"}}]}], "fileheader": [{"field": "ChargeDetailID", "alias": "ChargeDetailID", "generated": 0, "position": 1, "type": "String"}, {"field": "Age", "alias": "Age", "generated": 0, "position": 2, "type": "real"}, {"field": "Gender_New", "alias": "Gender_New", "generated": 1, "position": 3, "type": "String"}, {"field": "DateOfEntry", "alias": "DateOfEntry", "generated": 0, "position": 4, "type": "String"}, {"field": "DateOfService", "alias": "DateOfService", "generated": 0, "position": 5, "type": "String"}, {"field": "ProviderID", "alias": "ProviderID", "generated": 0, "position": 6, "type": "String"}, {"field": "CPTCode", "alias": "CPTCode", "generated": 0, "position": 7, "type": "String"}, {"field": "pdiagnosis1", "alias": "pdiagnosis1", "generated": 0, "position": 8, "type": "String"}, {"field": "pdiagnosis2", "alias": "pdiagnosis2", "generated": 0, "position": 9, "type": "String"}, {"field": "DepartmentCategory", "alias": "DepartmentCategory", "generated": 0, "position": 10, "type": "String"}, {"field": "SpecialityCategory", "alias": "SpecialityCategory", "generated": 0, "position": 11, "type": "String"}, {"field": "days_to_first_post", "alias": "days_to_first_post", "generated": 0, "position": 12, "type": "real"}, {"field": "PayerId", "alias": "PayerId", "generated": 0, "position": 13, "type": "String"}, {"field": "Age_Replaced", "alias": "Age_Replaced", "generated": 1, "position": 14, "type": "real"}, {"field": "Gender_Replaced", "alias": "Gender_Replaced", "generated": 1, "position": 15, "type": "String"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***WRITING DATAFRAME***

In [ ]:
#%run datapipetest1Hooks.ipynb
try:
	#sinkPreExecutionHook()

	dbfssink = DBFSConnector.put(df, "{'url': '/Surekha_DataPipe/undefined.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'is_header': 'Use Header Line', 'source_attributes': {'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Demodb2020', 'optionalDB': 'Demodb2020', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT sub2.ChargeDetailID,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,\r\n                CASE WHEN sub2.pdiagnosis1 IS NOT NULL THEN sub2.pdiagnosis1 ELSE 'XXXX' END as pdiagnosis1,\r\n                CASE WHEN sub2.pdiagnosis2 IS NOT NULL THEN sub2.pdiagnosis2 ELSE 'XXXX' END as pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,\r\n\t\t\t\tDATEDIFF(day, MIN(sub2.Chargedate), MIN(sub1.PaymentDate)) as days_to_first_post,\r\n\t\t\t\tSub2.PayerId\r\nFROM \r\n(\r\n\t\tSELECT \r\n\t\tDISTINCT cd.ChargeDetailID, Transactiondate as Chargedate,\r\n\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t\tp.Gender, cd.DateOfEntry, cd.DateOfService,\r\n\t\tpp.ProviderID,\r\n\t\tcptc.CPTCode,\r\n\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\td.DepartmentCategory,\r\n\t\ts.SpecialityCategory,\r\n\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID\r\n\t\tfrom Transactions t\r\n\t\tInner join chargedetail cd on t.Chargedetailid = cd.chargedetailid\r\n\t\tleft join transactiontype tt on t.TransactionTypeID=tt.TransactionTypeID\r\n\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\tWHERE TransactiontypeCategory = 'Charges'\r\n\t\tand t.DateOfEntry between '12/1/2020' and '12/31/2020'  \r\n) as sub2\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, TransactionDate as PaymentDate,\r\n\t\t\t t.DateOfEntry,\r\n\t\t\t count(*) as total_transactions\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tWHERE t.Amount > 0.1 \r\n\t\tand tt.TransactionTypeCategory = 'Payments' and transactiondate is not null \r\n\t\tgroup by t.DateOfEntry, t.ChargeDetailID,TransactionDate\r\n) as sub1\r\n on sub1.cid = sub2.ChargeDetailID\r\n where sub2.Gender is not NULL\r\n  Group by sub2.ChargeDetailID,  ChargeDate,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,sub2.pdiagnosis1, sub2.pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,sub1.PaymentDate,\r\n\t\t\t\tSub2.PayerId", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}, 'dbfsFileName': 'Datapipetest1', 'directory_path': '/Surekha_DataPipe'}")
	#sinkPostExecutionHook(dbfssink)

except Exception as ex: 
	logging.error(ex)
